In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
from pathlib import Path
import re 

import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid

root_path = Path('/dev/projects/cds/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
from api.data.base import data_path
from api.data.curves import curves

In [3]:
# Technically, the model requires the discount factor to be 1 at spot;
# but we can't do that and also have the discount curve extend back to
# the trade date.  For the time being, we'll keep discount = 1 at trade.
# The results match anyway.

#### Settings

In [4]:
cds_name = "TURKEY CDS USD SR 5Y D14 Curncy"
trade_date = pd.Timestamp('2021-04-28') # pd.Timestamp.today().floor('d')
curve_name = 'USD ISDA CDS SOFR Rates Swap Curve'
date_start = pd.Timestamp('2021-04-28')
date_end = pd.Timestamp('2022-06-23')

dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

In [5]:
### Bloomberg Connection
bq = BlpQuery(timeout=50000).start()

#### Local data

In [8]:
curve_hist = pd.read_csv(data_path / 'curve_history.csv', parse_dates=['date'], dayfirst=True)
cds_ref = pd.read_csv(data_path / 'cds_reference.csv', parse_dates=['maturity'], dayfirst=True)
cds_hist = pd.read_csv(data_path / 'cds_history.csv', parse_dates=['date'], dayfirst=True)

In [32]:
cds_ref_ = cds_ref.query(f"name == '{cds_name}'").squeeze()
cds_hist_ = cds_hist.query(f"name == '{cds_name}'").set_index('date').dropna().loc[date_start:date_end]
ch = cds_hist.query(f"name == '{cds_name}' and date == '{trade_date:{dt_fmt}}'").squeeze()

#### Functions

In [33]:
curve = [curve for curve in curves if curve['name'] == curve_name][0]
curve_hist_ = curve_hist.query(f'curve_name == "{curve["name"]}"')
curve_hist_ = curve_hist_.assign(
    tenor=pd.Categorical(
        curve_hist_["tenor"],
        categories=[memb['tenor'] for memb in curve['members']],
        ordered=True
    )
)
curve_hist_date_ = curve_hist_.query(f'date == "{trade_date:{dt_fmt}}"').sort_values('tenor')

In [34]:
def get_cds_implied_price(cds_name, dates, spreads):
    if isinstance(dates, pd.DatetimeIndex):
        dates = dates.to_series()
    if isinstance(dates, pd.Series):
        dates = dates.dt.strftime(bbg_dt_fmt).to_list()
    if isinstance(spreads, pd.Series):
        spreads = spreads.to_list()
        
    cds_implied_prices = bq.bdph(
        securities=[cds_name],
        fields=["CDS_QUOTED_PRICE"],
        id_override=dates,
        id_override_field="SW_CURVE_DT",
        CDS_FLAT_SPREAD=spreads,
        id_date=True
    )
    cds_implied_prices = cds_implied_prices.rename({'SW_CURVE_DT': 'date', 'CDS_QUOTED_PRICE': 'cds_implied_price'},
                                                   axis=1)
    return cds_implied_prices

In [35]:
def get_ql_date(date):
    return ql.Date(date.day, date.month, date.year)

In [36]:
def get_term_date(date):
    y, m, d = date.year, date.month, date.day
    day = 20
    if date < pd.Timestamp(year=y, month=3, day=20):
        year = y - 1
        month = 12
    elif date < pd.Timestamp(year=y, month=9, day=20):
        year = y
        month = 6
    else:
        year = y
        month = 12
    start = pd.Timestamp(year=year, month=month, day=day)
    return start + pd.tseries.offsets.DateOffset(years=5)

#### CDS

In [38]:
cds_ip = get_cds_implied_price(
    cds_name=cds_name,
    dates=cds_hist_.index,
    spreads=cds_hist_['last_price']
)

In [44]:
def get_cds(
    cds_name,
    curve_name,
    trade_date,
    notional=10000000,
    coupon=100
):
    X
    cds_ref_ = cds_ref.query(f"name == '{cds_name}'").squeeze()
    cds_hist_ = cds_hist.query(f"name == '{cds_name}' and date == '{trade_date:{dt_fmt}}'").squeeze()
    curve = [curve for curve in curves if curve['name'] == curve_name][0]
    curve_hist_ = curve_hist.query(f'curve_name == "{curve["name"]}"')
    curve_hist_ = curve_hist_.assign(
        tenor=pd.Categorical(
            curve_hist_["tenor"],
            categories=[memb['tenor'] for memb in curve['members']],
            ordered=True
        )
    )
    curve_hist_date_ = curve_hist_.query(f'date == "{trade_date:{dt_fmt}}"').sort_values('tenor')
    
    currency = cds_ref_['currency']
    
    deposit_tenors = [
        '1M', 
        '2M', 
        '3M', 
        '6M', 
        '1Y'
    ]
    swap_tenors = [
        '2Y', 
        '3Y', 
        '4Y', 
        '5Y', 
        '6Y',
        '7Y',
        '8Y',
        '9Y',
        '10Y',
        '12Y',
        '15Y',
        '20Y',
        '25Y',
        '30Y'
    ]
    
    deposit_data = curve_hist_date_.query(f"tenor in {deposit_tenors}").set_index('tenor')['last_price']
    swap_data = curve_hist_date_.query(f"tenor in {swap_tenors}").set_index('tenor')['last_price']
    # date = ql.Date(20, 6, 2022)
    ql_trade_date = get_ql_date(trade_date)
    ql_term_date = get_ql_date(get_term_date(trade_date))
    ql_upfront_date = ql.WeekendsOnly().advance(ql_trade_date, 3*ql.Period(ql.Daily))
    spot_date = ql.WeekendsOnly().advance(ql_trade_date, 2*ql.Period(ql.Daily))
    period_parser = ql.PeriodParser()
    ql.Settings.instance().setEvaluationDate(ql_trade_date)
    
    deposit_rate_helpers = [
        ql.DepositRateHelper(
            rate / 100,  # rate
            period_parser.parse(tenor),  # tenor                                      
            2,  # fixingDays 
            ql.WeekendsOnly(),
            ql.ModifiedFollowing,
            False, 
            ql.Actual360()
        ) for tenor, rate in deposit_data.items()
    ]

    isda_ibor = ql.IborIndex(
        'IsdaIbor',  # familyName
        3*ql.Period(ql.Monthly),  # tenor 
        2,  # settlementDays
        ql.USDCurrency(),  # currency
        ql.WeekendsOnly(),  # fixingCalendar
        ql.ModifiedFollowing, # convention
        False, # endOfMonth
        ql.Actual360(),  # dayCounter
    )

    swap_rate_helpers = [
        ql.SwapRateHelper(
            rate / 100,  # rate
            period_parser.parse(tenor),  # tenor
            ql.WeekendsOnly(),  # calendar
            ql.Semiannual,  # fixedFrequency
            ql.ModifiedFollowing,  # fixedConvention
            ql.Thirty360(),  # fixedDayCount
            isda_ibor,  # iborIndex
        ) for tenor, rate in swap_data.items()
    ]

    isda_rate_helpers = [*deposit_rate_helpers, *swap_rate_helpers]
    rate_curve = ql.PiecewiseLogLinearDiscount(ql_trade_date, isda_rate_helpers, ql.Actual365Fixed())
    discount_curve = ql.YieldTermStructureHandle(rate_curve)
    
    cds_schedule = ql.Schedule(
        ql_trade_date, 
        ql_term_date,
        3*ql.Period(ql.Monthly),
        ql.WeekendsOnly(),
        ql.Following, 
        ql.Unadjusted,
        ql.DateGeneration.CDS,
        False
    )
    
    spread = cds_hist_['last_price']
    quoted_trade = ql.CreditDefaultSwap(
        ql.Protection.Buyer,  # side
        notional,  # nominal
        0,  # upfront
        spread / 10000,  # spread
        cds_schedule,  # Schedule
        ql.Following, 
        ql.Actual360(), 
        True, 
        True, 
        ql_trade_date,
        ql_upfront_date, 
        ql.FaceValueClaim(), 
        ql.Actual360(True),
        True
    )

    h = quoted_trade.impliedHazardRate(
        0, 
        discount_curve, 
        ql.Actual365Fixed(),
        cds_ref_['recovery'], 
        1e-10,
        ql.CreditDefaultSwap.ISDA
    )
    fhr_curve = ql.FlatHazardRate(
        0, 
        ql.WeekendsOnly(),
        ql.QuoteHandle(ql.SimpleQuote(h)),
        ql.Actual365Fixed()
    )
    pd_curve = ql.DefaultProbabilityTermStructureHandle(fhr_curve)

    engine = ql.IsdaCdsEngine(pd_curve, cds_ref_['recovery'], discount_curve)

    conventional_trade = ql.CreditDefaultSwap(
        ql.Protection.Buyer,  # side
        notional,  # notional
        0,  # upfront
        coupon / 10000,  # spread
        cds_schedule,  # schedule
        ql.Following,  # paymentConvention
        ql.Actual360(),  # dayCounter
        True,  # settlesAccrual
        True,  # paysAtDefaultTime
        ql_trade_date,  # protectionStart
        ql_upfront_date,  # upfrontDate
        ql.FaceValueClaim(),  # Claim
        ql.Actual360(True),  # lastPeriodDayCounter
        True,  # rebatesAccrual
    )
    conventional_trade.setPricingEngine(engine)
    return conventional_trade

In [48]:

cdss = {}
up = []
for date in cds_hist_.index:
    cds = get_cds(
    cds_name=cds_name,
    curve_name=curve_name,
    trade_date=date
)
    
    # cdss[date] = cds
    up.append({'up': 100 * (1 + cds.fairUpfront()), 'npv': cds.NPV()})

In [50]:
pd.DataFrame(up, index=cds_hist_.index)

up           npv
date                               
2021-04-28  85.461688  1.453828e+06
2021-04-29  85.712940  1.428704e+06
2021-04-30  85.701344  1.429862e+06
2021-05-04  85.647556  1.435242e+06
2021-05-05  85.914062  1.408590e+06
...               ...           ...
2022-06-17  73.774738  2.621978e+06
2022-06-20  73.800516  2.619619e+06
2022-06-21  74.404768  2.559206e+06
2022-06-22  74.457848  2.553680e+06
2022-06-23  74.517067  2.547770e+06

[294 rows x 2 columns]

In [51]:
cds_hist_.loc[date]

security                      CTURK1U5 Curncy
last_price                              798.0
name          TURKEY CDS USD SR 5Y D14 Curncy
Name: 2022-06-23 00:00:00, dtype: object

In [21]:
date

Timestamp('2021-05-03 00:00:00', freq='B')

In [15]:
cds = get_cds(
    cds_name=cds_name,
    curve_name=curve_name,
    trade_date=trade_date
)

In [16]:
100 * (1 + cds.fairUpfront())

85.46168755684171

In [18]:
cds_ip

date                         security  cds_implied_price
0   2021-04-28  TURKEY CDS USD SR 5Y D14 Curncy          85.538564
1   2021-04-29  TURKEY CDS USD SR 5Y D14 Curncy          85.786698
2   2021-04-30  TURKEY CDS USD SR 5Y D14 Curncy          85.774157
3   2021-05-03  TURKEY CDS USD SR 5Y D14 Curncy          85.792018
4   2021-05-04  TURKEY CDS USD SR 5Y D14 Curncy          85.715741
..         ...                              ...                ...
297 2022-06-17  TURKEY CDS USD SR 5Y D14 Curncy          73.782945
298 2022-06-20  TURKEY CDS USD SR 5Y D14 Curncy          73.810974
299 2022-06-21  TURKEY CDS USD SR 5Y D14 Curncy          74.415039
300 2022-06-22  TURKEY CDS USD SR 5Y D14 Curncy          74.468074
301 2022-06-23  TURKEY CDS USD SR 5Y D14 Curncy          74.527514

[302 rows x 3 columns]

In [ ]:
cds.NPV()

In [ ]:
curve_hist_date_

In [ ]:
ch

In [ ]:
cds_ref_

In [ ]:
pd.read_csv('cip.csv')

In [ ]:
cd_ip.to_csv('cip.csv', index=False)

#### QuantLib

In [ ]:
# date = ql.Date(20, 6, 2022)
ql_trade_date = get_ql_date(trade_date)
ql_term_date = get_ql_date(get_term_date(trade_date))
ql_upfront_date = ql.WeekendsOnly().advance(ql_trade_date, 3*ql.Period(ql.Daily))
spot_date = ql.WeekendsOnly().advance(ql_trade_date, 2*ql.Period(ql.Daily))
period_parser = ql.PeriodParser()
ql.Settings.instance().setEvaluationDate(ql_trade_date)

In [ ]:
deposit_rate_helpers = [
    ql.DepositRateHelper(
        rate / 100,  # rate
        period_parser.parse(tenor),  # tenor                                      
        2,  # fixingDays 
        ql.WeekendsOnly(),
        ql.ModifiedFollowing,
        False, 
        ql.Actual360()
    ) for tenor, rate in deposit_data.items()
]

In [ ]:
isda_ibor = ql.IborIndex(
    'IsdaIbor',  # familyName
    3*ql.Period(ql.Monthly),  # tenor 
    2,  # settlementDays
    ql.USDCurrency(),  # currency
    ql.WeekendsOnly(),  # fixingCalendar
    ql.ModifiedFollowing, # convention
    False, # endOfMonth
    ql.Actual360(),  # dayCounter
)

swap_rate_helpers = [
    ql.SwapRateHelper(
        rate / 100,  # rate
        period_parser.parse(tenor),  # tenor
        ql.WeekendsOnly(),  # calendar
        ql.Semiannual,  # fixedFrequency
        ql.ModifiedFollowing,  # fixedConvention
        ql.Thirty360(),  # fixedDayCount
        isda_ibor,  # iborIndex
    ) for tenor, rate in swap_data.items()
]

isda_rate_helpers = [*deposit_rate_helpers, *swap_rate_helpers]
rate_curve = ql.PiecewiseLogLinearDiscount(ql_trade_date, isda_rate_helpers, ql.Actual365Fixed())
discount_curve = ql.YieldTermStructureHandle(rate_curve)

In [ ]:
cds_schedule = ql.Schedule(
    ql_trade_date, 
    ql_term_date,
    3*ql.Period(ql.Monthly),
    ql.WeekendsOnly(),
    ql.Following, 
    ql.Unadjusted,
    ql.DateGeneration.CDS,
    False
)

quoted_trade = ql.CreditDefaultSwap(
    ql.Protection.Buyer,  # side
    10000000,  # nominal
    0,  # upfront
    cds_price['last_price'] / 10000,  # spread
    cds_schedule,  # Schedule
    ql.Following, 
    ql.Actual360(), 
    True, 
    True, 
    ql_trade_date,
    ql_upfront_date, 
    ql.FaceValueClaim(), 
    ql.Actual360(True)
)
# quoted_trade.setPricingEngine(engine)

h = quoted_trade.impliedHazardRate(
    0, 
    discount_curve, 
    ql.Actual365Fixed(),
    cds_ref['recovery'], 
    1e-10,
    ql.CreditDefaultSwap.ISDA
)

In [ ]:
fhr_curve = ql.FlatHazardRate(
    ql_trade_date, 
    ql.WeekendsOnly(),
    ql.QuoteHandle(ql.SimpleQuote(h)),
    ql.Actual365Fixed()
)
pd_curve = ql.DefaultProbabilityTermStructureHandle(fhr_curve)

engine = ql.IsdaCdsEngine(pd_curve, cds_ref['recovery'], discount_curve)

conventional_trade = ql.CreditDefaultSwap(
    ql.Protection.Buyer,  # side
    10000000,  # notional
    0,  # upfront
    100 / 10000,  # spread
    cds_schedule,  # schedule
    ql.Following,  # paymentConvention
    ql.Actual360(),  # dayCounter
    True,  # settlesAccrual
    True,  # paysAtDefaultTime
    ql_trade_date,  # protectionStart
    ql_upfront_date,  # upfrontDate
    ql.FaceValueClaim(),  # Claim
    ql.Actual360(True),  # lastPeriodDayCounter
    True,  # rebatesAccrual
)
conventional_trade.setPricingEngine(engine)

In [ ]:
f"NPV: {conventional_trade.NPV()}, Price: {100 * (1+conventional_trade.fairUpfront())}"

In [ ]:
quoted_trade.coupons()

In [ ]:
ql.CashFlow.amount(conventional_trade.coupons())

In [ ]:
conventional_trade.couponLegNPV()

In [ ]:
conventional_trade.defaultLegNPV()

In [ ]:
conventional_trade.NPV()

In [ ]:
conventional_trade.accrualRebateNPV()

In [ ]:
1+conventional_trade.fairUpfront()

In [ ]:
quoted_trade.fairUpfront()


In [ ]:
probabilityCurve = ql.RelinkableDefaultProbabilityTermStructureHandle()

In [ ]:

termDate = ql.Date(20, 6, 2010),
             ql.Date(20, 6, 2011),
             ql.Date(20, 6, 2012),
             ql.Date(20, 6, 2016),
             ql.Date(20, 6, 2019)

spreads = [0.001, 0.1]
recoveries = [0.2, 0.4]

markitValues = [97798.29358,  # 0.001
                97776.11889,  # 0.001
                -914971.5977,  # 0.1
                -894985.6298,  # 0.1
                186921.3594,  # 0.001
                186839.8148,  # 0.001
                -1646623.672,  # 0.1
                -1579803.626,  # 0.1
                274298.9203,
                274122.4725,
                -2279730.93,
                -2147972.527,
                592420.2297,
                591571.2294,
                -3993550.206,
                -3545843.418,
                797501.1422,
                795915.9787,
                -4702034.688,
                -4042340.999]

tolerance = 1.0e-2

l = 0
distance = 0

data = []
upfront_date = ql.WeekendsOnly().advance(trade_date, 3 * ql.Period(ql.Daily))
for termDate in termDates:
    for spread in spreads:
        for recovery in recoveries:
            cdsSchedule = ql.Schedule(
                trade_date, 
                termDate,
                3 * ql.Period(ql.Monthly),
                ql.WeekendsOnly(),
                ql.Following, 
                ql.Unadjusted,
                ql.DateGeneration.CDS,
                False
            )

            quotedTrade = ql.CreditDefaultSwap(
                ql.Protection.Buyer, 
                10000000, 
                0, 
                spread, 
                cdsSchedule,
                ql.Following, ql.Actual360(), True, True, trade_date,
                upfront_date, ql.FaceValueClaim(), ql.Actual360(True))

            h = quotedTrade.impliedHazardRate(
                0, 
                discountCurve, 
                ql.Actual365Fixed(),
                recovery, 
                1e-10,
                ql.CreditDefaultSwap.ISDA
            )

            probabilityCurve.linkTo(
                ql.FlatHazardRate(0, ql.WeekendsOnly(),
                                  ql.QuoteHandle(ql.SimpleQuote(h)),
                                  ql.Actual365Fixed()))

            engine = ql.IsdaCdsEngine(probabilityCurve, recovery, discountCurve)
            conventionalTrade = ql.CreditDefaultSwap(
                ql.Protection.Buyer, 10000000, 0, 0.01, cdsSchedule,
                ql.Following, ql.Actual360(), True, True, trade_date,
                upfront_date, ql.FaceValueClaim(), ql.Actual360(True))
            conventionalTrade.setPricingEngine(engine)

            upfront = conventionalTrade.notional() * conventionalTrade.fairUpfront()

            data.append(
                (termDate,
                 spread,
                 recovery,
                 h,
                 upfront,
                 markitValues[l],
                 abs(upfront - markitValues[l]),
                 abs(upfront - markitValues[l]) < tolerance)
            )
            distance = distance + abs(upfront - markitValues[l])

            l = l + 1

df = pd.DataFrame(data, columns=["Term date", "Spread", "Recovery",
                                 "Hazard rate", "Upfront", "Markit value", "Distance", "Within tolerance"])


df.style.format({'Spread': '{:.4%}', 'Hazard rate': '{:.2%}', 'Upfront': '{:.2f}',
                 'Markit value': '{:.2f}', 'Distance': '{:.6f}'})

In [ ]:
probabilityCurve = ql.RelinkableDefaultProbabilityTermStructureHandle()

termDates = [ql.Date(20, 6, 2010),
             ql.Date(20, 6, 2011),
             ql.Date(20, 6, 2012),
             ql.Date(20, 6, 2016),
             ql.Date(20, 6, 2019)]

spreads = [0.001, 0.1]
recoveries = [0.2, 0.4]

markitValues = [97798.29358,  # 0.001
                97776.11889,  # 0.001
                -914971.5977,  # 0.1
                -894985.6298,  # 0.1
                186921.3594,  # 0.001
                186839.8148,  # 0.001
                -1646623.672,  # 0.1
                -1579803.626,  # 0.1
                274298.9203,
                274122.4725,
                -2279730.93,
                -2147972.527,
                592420.2297,
                591571.2294,
                -3993550.206,
                -3545843.418,
                797501.1422,
                795915.9787,
                -4702034.688,
                -4042340.999]

tolerance = 1.0e-2

l = 0
distance = 0

data = []
upfront_date = ql.WeekendsOnly().advance(trade_date, 3 * ql.Period(ql.Daily))
for termDate in termDates:
    for spread in spreads:
        for recovery in recoveries:
            cdsSchedule = ql.Schedule(
                trade_date, 
                termDate,
                3 * ql.Period(ql.Monthly),
                ql.WeekendsOnly(),
                ql.Following, 
                ql.Unadjusted,
                ql.DateGeneration.CDS,
                False
            )

            quotedTrade = ql.CreditDefaultSwap(
                ql.Protection.Buyer, 
                10000000, 
                0, 
                spread, 
                cdsSchedule,
                ql.Following, ql.Actual360(), True, True, trade_date,
                upfront_date, ql.FaceValueClaim(), ql.Actual360(True))

            h = quotedTrade.impliedHazardRate(
                0, 
                discountCurve, 
                ql.Actual365Fixed(),
                recovery, 
                1e-10,
                ql.CreditDefaultSwap.ISDA
            )

            probabilityCurve.linkTo(
                ql.FlatHazardRate(0, ql.WeekendsOnly(),
                                  ql.QuoteHandle(ql.SimpleQuote(h)),
                                  ql.Actual365Fixed()))

            engine = ql.IsdaCdsEngine(probabilityCurve, recovery, discountCurve)
            conventionalTrade = ql.CreditDefaultSwap(
                ql.Protection.Buyer, 10000000, 0, 0.01, cdsSchedule,
                ql.Following, ql.Actual360(), True, True, trade_date,
                upfront_date, ql.FaceValueClaim(), ql.Actual360(True))
            conventionalTrade.setPricingEngine(engine)

            upfront = conventionalTrade.notional() * conventionalTrade.fairUpfront()

            data.append(
                (termDate,
                 spread,
                 recovery,
                 h,
                 upfront,
                 markitValues[l],
                 abs(upfront - markitValues[l]),
                 abs(upfront - markitValues[l]) < tolerance)
            )
            distance = distance + abs(upfront - markitValues[l])

            l = l + 1

df = pd.DataFrame(data, columns=["Term date", "Spread", "Recovery",
                                 "Hazard rate", "Upfront", "Markit value", "Distance", "Within tolerance"])


df.style.format({'Spread': '{:.4%}', 'Hazard rate': '{:.2%}', 'Upfront': '{:.2f}',
                 'Markit value': '{:.2f}', 'Distance': '{:.6f}'})

In [ ]:
print('total distance:', distance)